In [1]:
import pandas as pd
import numpy as np
import math
import copy

In [2]:
rating_df_columns = ["UserId", "MovieId", "Rating", "TimeStamp"]
rating_df = pd.read_table("data_set/ratings.dat", sep="::", engine = 'python', names=rating_df_columns)
rating_df.drop(rating_df.index[1000000:1000209], inplace=True)

In [3]:
rating_matrix = np.asarray(rating_df.pivot(index = "UserId", columns = "MovieId", values = "Rating").fillna(0))

In [4]:
def svd(A):
    transpose_flag = 0
    if A.shape[0] > A.shape[1]:  
        transpose_flag = 1
        A = A.T
    
    #Convert matrix A into square matrix to be able to perform eigen value decomp
    AAt = np.matmul(A, A.T)  #get the value of U(EE.T)U.T.
    
    AtA = np.matmul(A.T, A) #get the value of value of V(EE.T)V.T
    
    #getting the eigen values and vectors for both the square matrices
    eval_AAt, evec_AAt = np.linalg.eig(AAt) #use eigen value decomposition to get the the eigen value and the eigen vector. In this case it is V
    eval_AtA, evec_AtA = np.linalg.eig(AtA) # in this case it is U.
    
    #print(eval_AAt)
    evec_AAt_transpose = evec_AAt.T #get the transpose of eigen vectors. In this case it is U.T
    evec_AtA_transpose = evec_AtA.T #in this case it is V.T
    
    #ignore the vectors corresponding to negative eigen values
    eval_AAt[eval_AAt < 10e-2] = 0
    eval_AtA[eval_AtA < 10e-2] = 0
    #print(eval_AAt)
    #print(eval_AAt.shape)
    
    eval_AAt = np.sqrt(eval_AAt.real) #get the sqrt of all elements of the eigen value matrix as sigma^2 = lambda.
    eval_AtA = np.sqrt(eval_AtA.real)
    
    evec_AAt = evec_AAt.real #take only the real values of eigen vectors
    evec_AtA = evec_AtA.real
    
    argsort_eval_AAt = np.argsort(-eval_AAt) #Sort the values in the descending order but keep the value of their original positions
    argsort_eval_AtA = np.argsort(-eval_AtA)
    
    U = np.zeros(evec_AAt.shape)
    sigma1 = np.sort(eval_AAt)[::-1] #sort the eigen values in descending order.
    
    V = np.zeros(evec_AtA.shape)
    sigma2 = (np.sort(eval_AtA))[::-1] #sort the eigen values in descending order.
    
    for i,j in enumerate(argsort_eval_AtA):
        V[:,i] = evec_AtA[:,j]  #every element of the eigen value corresponds to a specific evector. Use that cor
        
    #sigma = np.diag(sigma1)
    #print(sigma)
    sigma = np.zeros(A.shape)
    sigma[:, :A.shape[0]] = np.diag(sigma1) #set the eigen value matrix as a diagonal matrix
       
    for i in range(U.shape[1]):
        if sigma1[i] != 0:
            U[:,i] = (A.dot(V[:,i]))/sigma1[i]
        else:
            U[:,i] = 0
        
    if transpose_flag == 0:
        return U, sigma, V.T
    else:
        return V, sigma.T, U.T

In [5]:
A = rating_matrix
print(A.shape)

(6040, 3706)


In [6]:
#calculate the probabilities for column sampling
prob = A.T.dot(A)
column_len = np.asarray(np.diagonal(prob))
denom = np.abs(column_len).sum(axis = 0)
column_len = column_len/denom


#calculate the probabilities for row sampling
prob1 = A.dot(A.T)
row_len = np.asarray(np.diagonal(prob1))
denom = np.abs(row_len).sum(axis = 0)
row_len= row_len/denom


In [7]:
#here k is 884, therefore number of columns is 4times 884 and randomly chooose the column numbers
columns = 3536
rows = 3536
np.random.seed(201)
selected_columns = np.random.choice(range(column_len.shape[0]), size = (columns), replace = True, p = column_len)
#print(selected_columns)
selected_rows = np.random.choice(range(row_len.shape[0]), size = (rows), replace = True, p = row_len)

In [8]:
selected_rows.sort()
selected_columns.sort()

In [9]:
C = np.zeros((A.shape[0], columns))
R = np.zeros((rows, A.shape[1]))
W = np.zeros((rows, columns))

In [10]:
#fill the C and R matrix according to the selected column values and row values
for i,j in enumerate(selected_columns):
    C[:,i] = A[:,j]/(np.sqrt(columns * column_len[j]))
    
for i,j in enumerate(selected_rows):
    R[i,:] = A[j,:]/(np.sqrt(rows * row_len[j]))

In [11]:
#build the W matrix using the intersection of C and R
for i,j in enumerate(selected_rows):
    for k,l in enumerate(selected_columns):
        W[i,k] = A[j,l]

In [12]:
X, Z, Y_transpose = svd(W)

In [13]:
W_pred = X.dot(Z.dot(Y_transpose))

In [14]:
np.sqrt(((W_pred - W) ** 2).sum()/ (W.shape[0] * W.shape[1]))

3.6821383718475975e-12

In [15]:
Z_plus = np.zeros(Z.shape)
for i in range(Z.shape[0]):
    for j in range(Z.shape[1]):
        if Z[i][j] != 0:
            Z_plus[i][j] = 1/Z[i][j]

In [16]:
U = (Y_transpose.T).dot((Z_plus.T).dot(X.T))

In [17]:
A_pred = C.dot(U.dot(R))

In [18]:
rmse = np.sqrt( ((A_pred - A)**2).sum()/ (A.shape[0]*A.shape[1]) )
print(rmse)

1.19827587395
